# Web Crawling

In [1]:
import FinanceDataReader as fdr
import pandas as pd
from bs4 import BeautifulSoup
import datetime as dt
from datetime import datetime,timedelta
from urllib.request import urlopen
import sqlalchemy 
import pymysql
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

# MoneyTrend

In [ ]:
def get_money_trend():
    
    url = 'http://finance.naver.com/sise/sise_deposit.nhn?&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('&')[1]
    last = last.split('=')[1]
    last = int(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\moneytrend.xlsx'   
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'고객예탁금': [],'신용잔고': [],'주식형 펀드': [],'혼합형 펀드': [],'채권형 펀드': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['고객예탁금','신용잔고','주식형 펀드','혼합형 펀드','채권형 펀드']


    # count mask
    mask = [1,3,5,7,9]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','rate_down','rate_up']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    date_list.append(date_)
                        
                      
                elif count in mask:
                    temp = int((count-1)/2)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
        
                count += 1
            if len(dictionary['고객예탁금']) != len(dictionary['채권형 펀드']):
                print(str(i)+ '번째 페이지에서 누락된 값 발생')
                print('누락된 데이터를 제거합니다')
                    
                date_list.pop(-1)
                dictionary['고객예탁금'].pop(-1)
                dictionary['신용잔고'].pop(-1)
                dictionary['주식형 펀드'].pop(-1)
                dictionary['혼합형 펀드'].pop(-1)
                
    # 개별 list 요소 갯수 파악 
    #print(len(date_list))
    #print(len(dictionary['고객예탁금']))
    #print(len(dictionary['신용잔고']))
    #print(len(dictionary['주식형 펀드']))
    #print(len(dictionary['혼합형 펀드']))
    #print(len(dictionary['채권형 펀드']))
    print(str(i) + '번째 페이지 크롤링 완료')
    df = pd.DataFrame(dictionary,index = date_list)
    df = df.sort_index()
    df.to_excel(path, encoding='utf-8')
    print(df)


get_money_trend()



In [ ]:
def get_money_trend_date():
    url = 'http://finance.naver.com/sise/sise_deposit.nhn?&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('&')[1]
    last = last.split('=')[1]
    last = int(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\money_trend.xlsx'

    
    until_date = input("날짜를 입력하세요 sample: '2019-01-10': ")
    
    year = until_date.split('-')[0]
    mm = until_date.split('-')[1]
    dd = until_date.split('-')[2]
    year=year[2:]
    until_date = year+'-'+mm+'-'+dd
    
    #df = DataFrame(columns = ['고객예탁금', '신용잔고','주식형 펀드','혼합형 펀드','채권형 펀드'])

    # 날짜를 받을 리스트
    date_list = []

    
    # 값을 받을 사전
    dictionary = {'고객예탁금': [],
                  '신용잔고': [],
                  '주식형 펀드': [],
                  '혼합형 펀드': [],
                  '채권형 펀드': []
                  }

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['고객예탁금','신용잔고','주식형 펀드','혼합형 펀드','채권형 펀드']


    # count mask
    mask = [1,3,5,7,9]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','rate_down','rate_up']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    if date_ <=  until_date :
                    #if date_ <=  '19-03-05' :
                        df = pd.DataFrame(dictionary,index = date_list)
                        df = df.sort_index()
                        df.to_excel(path, encoding='utf-8')
                        return df
                    date_list.append(date_)
                    
                elif count in mask:
                    temp = int((count-1)/2)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
                
       
                count += 1

get_money_trend_date()

# KPI200

In [ ]:
def get_kpi200():
    
    url = 'https://finance.naver.com/sise/sise_index_day.nhn?code=KPI200&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\kpi200.xlsx'
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'KPI200': [],'거래량': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['KPI200','거래량']


    # count mask
    mask = [1,3]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','number_1']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    date_list.append(date_)
                        
                      
                elif count in mask:
                    temp = int(count/3)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
        
                count += 1
            if len(date_list) != len(dictionary['KPI200']):
                print(str(i)+ '번째 페이지에서 누락된 값 발생')
                print('누락된 데이터를 제거합니다')
                    
                date_list.pop(-1)
                #dictionary['KPI200'].pop(-1)
                dictionary['거래량'].pop(-1)
                
    # 개별 list 요소 갯수 파악 
    #print(len(date_list))
    #print(len(dictionary['개인']))
    #print(len(dictionary['외국인']))
    #print(len(dictionary['기관']))

    print(str(i) + '번째 페이지 크롤링 완료')
    df = pd.DataFrame(dictionary,index = date_list)
    df = df.sort_index()
    df.to_excel(path, encoding='utf-8')
    print(df)

get_kpi200()

In [ ]:
def get_kpi200_date():
    
    url = 'https://finance.naver.com/sise/sise_index_day.nhn?code=KPI200&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\kpi200.xlsx'

    until_date = input("날짜를 입력하세요 sample: '2019-01-10': ")
    
    year = until_date.split('-')[0]
    mm = until_date.split('-')[1]
    dd = until_date.split('-')[2]
    #year=year[2:]
    until_date = year+'-'+mm+'-'+dd
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'KPI200': [],'거래량': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['KPI200','거래량']


    # count mask
    mask = [1,3]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','number_1']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    if date_ <=  until_date :
                    #if date_ <=  '19-03-05' :
                        df = pd.DataFrame(dictionary,index = date_list)
                        df = df.sort_index()
                        df.to_excel(path, encoding='utf-8')
                        return df   
                    date_list.append(date_)
                    #print(date_list)
                elif count in mask:
                    temp = int(count/3)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
                    #print(dictionary[name_list[temp]])
                count += 1
                
get_kpi200_date()

# Investor Trend

In [ ]:
def get_investor_trend_date():
    
    url = 'http://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate=2020601&sosok=&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\investortrend.xlsx'

    until_date = input("날짜를 입력하세요 sample: '2019-01-10': ")
    
    year = until_date.split('-')[0]
    mm = until_date.split('-')[1]
    dd = until_date.split('-')[2]
    year=year[2:]
    until_date = year+'-'+mm+'-'+dd
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'개인': [],'외국인': [],'기관': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['개인','외국인','기관']


    # count mask
    mask = [1,2,3]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date2','rate_down3','rate_up3']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    if date_ <=  until_date :
                        df = pd.DataFrame(dictionary,index = date_list)
                        df = df.sort_index()
                        df.to_excel(path, encoding='utf-8')
                        return df   
                    date_list.append(date_)
                    #print(date_list)
                elif count in mask:
                    temp = int(count-1)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
                    
                count += 1
                
get_investor_trend_date()

# Program Trend

In [ ]:
def get_program_trend():
    
    url = 'https://finance.naver.com/sise/programDealTrendDay.nhn?bizdate=20200315&sosok=&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\programtrend.xlsx'
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'차익': [],'비차익': [],'전체': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['차익','비차익','전체']


    # count mask
    mask = [3,6,9]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','rate_down','rate_up']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    date_list.append(date_)
                        
                      
                elif count in mask:
                    temp = int((count/3)-1)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
        
                count += 1
            if len(date_list) != len(dictionary['전체']):
                print(str(i)+ '번째 페이지에서 누락된 값 발생')
                print('누락된 데이터를 제거합니다')
                    
                date_list.pop(-1)
                dictionary['차익'].pop(-1)
                dictionary['비차익'].pop(-1)
                #dictionary['전체'].pop(-1)
                
    # 개별 list 요소 갯수 파악 
    print(len(date_list))
    print(len(dictionary['차익']))
    print(len(dictionary['비차익']))
    print(len(dictionary['전체']))

    print(str(i) + '번째 페이지 크롤링 완료')
    df = pd.DataFrame(dictionary,index = date_list)
    df = df.sort_index()
    df.to_excel(path, encoding='utf-8')
    print(df)

get_program_trend()

In [ ]:
def get_program_trend_date():
    
    url = 'https://finance.naver.com/sise/programDealTrendDay.nhn?bizdate=20200315&sosok=&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\programtrend.xlsx'

    until_date = input("날짜를 입력하세요 sample: '2019-01-10': ")
    
    year = until_date.split('-')[0]
    mm = until_date.split('-')[1]
    dd = until_date.split('-')[2]
    year=year[2:]
    until_date = year+'-'+mm+'-'+dd
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'차익': [],'비차익': [],'전체': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['차익','비차익','전체']


    # count mask
    mask = [3,6,9]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','rate_down','rate_up']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    if date_ <=  until_date :
                        df = pd.DataFrame(dictionary,index = date_list)
                        df = df.sort_index()
                        df.to_excel(path, encoding='utf-8')
                        return df   
                    date_list.append(date_)
                    #print(date_list)
                elif count in mask:
                    temp = int((count/3)-1)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
                    
                count += 1
get_program_trend_date()

# 선물

In [ ]:
def  futures():
    
    url = 'https://finance.naver.com/sise/sise_index_day.nhn?code=FUT&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\futures.xlsx'
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'Close': [],'updown': [],'rate': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['Close','updown','rate']


    # count mask
    mask = [1,2,3]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date','number_1','rate_down']})
            count = 0
    
            for td in tds:
                if count == 0:
                    date_ = td.text.strip().replace('.','-')
                    date_list.append(date_)
                        
                      
                elif count in mask:
                    temp = int(count-1)
                    dictionary[name_list[temp]].append(td.text.strip().replace(',',''))
        
                count += 1
            if len(date_list) != len(dictionary['Close']):
                print(str(i)+ '번째 페이지에서 누락된 값 발생')
                print('누락된 데이터를 제거합니다')
                    
                date_list.pop(-1)
                dictionary['Close'].pop(-1)
                dictionary['updown'].pop(-1)
                dictionary['rate'].pop(-1)
                
    # 개별 list 요소 갯수 파악 
    #print(len(date_list))
    #print(len(dictionary['개인']))
    #print(len(dictionary['외국인']))
    #print(len(dictionary['기관']))

    print(str(i) + '번째 페이지 크롤링 완료')
    df = pd.DataFrame(dictionary,index = date_list)
    df = df.sort_index()
    df.to_excel(path, encoding='utf-8')
    print(df)
    
futures()

# 관리종목

In [9]:
from  datetime import datetime
import pandas as pd
import xlsxwriter

today = datetime.now()
today = today.strftime("%Y-%m-%d")
#today=input('입력')
url = 'http://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate=2020601&sosok=&page=1'
url = 'https://finance.naver.com/sise/management.nhn'
source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
data = []

path = 'd:\\stockdata\\관리종목\\'+today+'.xlsx'
body = source.find('body')
trs = body.find_all('tr')
name = []
for tr in trs:
    tds = tr.find_all('a',{'class':"tltle"})
    for td in tds:
        name.append(td.text.strip())

df = pd.DataFrame(name)
df['Date']=str(today)
df = df.set_index('Date')
df.columns=['Name']
df.to_excel(path)
print(df)

                  Name
Date                  
2019-09-07     제이테크놀로지
2019-09-07      한류AI센터
2019-09-07        경남제약
2019-09-07         캔서롭
2019-09-07     차이나그레이트
2019-09-07   이스트아시아홀딩스
2019-09-07      코오롱티슈진
2019-09-07      이매진아시아
2019-09-07        포스링크
2019-09-07         피앤텔
2019-09-07  대신밸런스제4호스팩
2019-09-07        레드로버
2019-09-07        파인넥스
2019-09-07        넷게임즈
2019-09-07     신한제3호스팩
2019-09-07          신한
2019-09-07          지코
2019-09-07       웅진에너지
2019-09-07         컨버즈
2019-09-07      세화아이엠씨
2019-09-07     폴루스바이오팜
2019-09-07        바른전자
2019-09-07         테라셈
2019-09-07      스튜디오썸머
2019-09-07        세원물산
2019-09-07       미래SCI
2019-09-07        디에스티
2019-09-07      에이아이비트
2019-09-07      크로바하이텍
2019-09-07          KD
...                ...
2019-09-07        에스마크
2019-09-07       코드네이처
2019-09-07      에스제이케이
2019-09-07       디지탈옵틱
2019-09-07         유테크
2019-09-07        아이엠텍
2019-09-07     에이치엘비파워
2019-09-07         옴니텔
2019-09-07       엔터메이트
2019-09-07       이에스브이
2019-09-07 

# Test

In [ ]:
def get_program_trend():
    
    url = 'https://finance.naver.com/sise/management.nhn'

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    body = source.find('body')
    trs = body.find_all('tr')
    #print(trs)
    
    name_list = []
    
    for tr in trs:
        tds = tr.find_all('td')
        for a in tds:
            #title = a.find_all('a',{'href':"/item/main.nhn"})
            title = a.find_all('a',{'class':"tltle"})
            #title = a.find_all('a')
            name_ = title.text.strip().replace('.','')
            #df = pd.DataFrame(dictionary,index = date_list)
            print(name_)
        #print(tds)

get_program_trend()

In [ ]:
def get_program_trend():
    
    url = 'https://finance.naver.com/sise/management.nhn'

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    body = source.find('body')
    trs = body.find_all('tr')
    #print(trs)
    
    name = []
    
    for tr in trs:
        tds = tr.find_all('td')
        #print(tds)
        for td in tds:
            td = td.find_all('a',{'class':"tltle"})
            name.append(td)
            #title = a.find_all('a',{'href':"/item/main.nhn"})
            #tltle = a.find_all('a',{'class':"tltle"})
            #title = a.find_all('a')
            #name = tltle.text
            #df = pd.DataFrame(dictionary,index = date_list)
            #print(name)
            #print(td)
        print(name)

get_program_trend()

In [72]:
from  datetime import datetime
import pandas as pd

today = datetime.now()
today = today.strftime("%Y-%m-%d")

#url = 'http://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate=2020601&sosok=&page=1'
url = 'https://finance.naver.com/sise/management.nhn'
source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
data = []

path = 'd:\\stockdata\\관리종목\\'+today+'.xlsx'
body = source.find('body')
trs = body.find_all('tr')
name = []
for tr in trs:
    #tds = tr.find_all('td',{'class':['date2','rate_down3','rate_up3']})
    tds = tr.find_all('a',{'class':"tltle"})
    for td in tds:
        name.append(td.text.strip())

df = pd.DataFrame(name,columns=['관리종목'])
df['Date']=today
df = df.set_index('Date')
df.to_excel(path)
print(df)


                  관리종목
Date                  
2019-09-07     제이테크놀로지
2019-09-07      한류AI센터
2019-09-07        경남제약
2019-09-07         캔서롭
2019-09-07     차이나그레이트
2019-09-07   이스트아시아홀딩스
2019-09-07      코오롱티슈진
2019-09-07      이매진아시아
2019-09-07        포스링크
2019-09-07         피앤텔
2019-09-07  대신밸런스제4호스팩
2019-09-07        레드로버
2019-09-07        파인넥스
2019-09-07        넷게임즈
2019-09-07     신한제3호스팩
2019-09-07          신한
2019-09-07          지코
2019-09-07       웅진에너지
2019-09-07         컨버즈
2019-09-07      세화아이엠씨
2019-09-07     폴루스바이오팜
2019-09-07        바른전자
2019-09-07         테라셈
2019-09-07      스튜디오썸머
2019-09-07        세원물산
2019-09-07       미래SCI
2019-09-07        디에스티
2019-09-07      에이아이비트
2019-09-07      크로바하이텍
2019-09-07          KD
...                ...
2019-09-07        에스마크
2019-09-07       코드네이처
2019-09-07      에스제이케이
2019-09-07       디지탈옵틱
2019-09-07         유테크
2019-09-07        아이엠텍
2019-09-07     에이치엘비파워
2019-09-07         옴니텔
2019-09-07       엔터메이트
2019-09-07       이에스브이
2019-09-07 

In [67]:
from datetime import datetime

today = datetime.now()
today = today.strftime("%Y-%m-%d")
print(today)

2019-09-07


In [ ]:
def get_investor_trend_date():
    
    url = 'http://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate=2020601&sosok=&page='

    source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
    source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

    last = source.find('td',class_='pgRR').find('a')['href']
    last = last.split('page')[1]
    last = last.split('=')[1]
    last = int(last)
    print(last)

    # 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
    path = 'd:\\investortrend.xlsx'

    until_date = input("날짜를 입력하세요 sample: '2019-01-10': ")
    
    year = until_date.split('-')[0]
    mm = until_date.split('-')[1]
    dd = until_date.split('-')[2]
    year=year[2:]
    until_date = year+'-'+mm+'-'+dd
    
    # 날짜를 받을 리스트
    date_list = []

    # 값을 받을 사전
    dictionary = {'개인': [],'외국인': [],'기관': []}

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['개인','외국인','기관']


    # count mask
    mask = [1,2,3]
    
    for i in range(1,last+1):
        
        source = urlopen(url+ str(i)).read()
        source = BeautifulSoup(source,'lxml')

        #tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})
        #trs = tbody.find_all('tr')

        body = source.find('body')
        trs = body.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td',{'class':['date2','rate_down3','rate_up3']})
            print(tds)
            #for td in tds:
                #print(td.fide_all('td',{'class':'date2'}))
                #print(td.text.strip())
get_investor_trend_date()

In [ ]:
get_investor_trend_date()